<a href="https://colab.research.google.com/github/aio25-mix002/m07-p7.1/blob/main/notebooks/runbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Checking

In [43]:
import os
from pathlib import Path

def notebook_runtime() -> str:
    # Kaggle
    if any(k in os.environ for k in ("KAGGLE_URL_BASE", "KAGGLE_KERNEL_RUN_TYPE", "KAGGLE_DATA_PROXY_TOKEN")) \
       or Path("/kaggle").exists():
        return "kaggle"

    # Google Colab
    if "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ or Path("/content").exists():
        return "colab"

    # Local / other Jupyter (VS Code, JupyterLab, etc.)
    return "local"

NOTEBOOK_RUNTIME = notebook_runtime()
print("Runtime:", NOTEBOOK_RUNTIME)


Runtime: kaggle


## Configuration

In [44]:
from dotenv import load_dotenv

SAVE_TO_REMOTE_STORAGE = True
ROOT_DIR = None
WORKING_DIR = None
CODE_DIR = None
INPUT_DATA_DIR = None
OUTPUT_DIR = None
APPCONFIG_ENV_FILENAME = None

if NOTEBOOK_RUNTIME == "kaggle":
    SAVE_TO_REMOTE_STORAGE = False
    ROOT_DIR = Path("/kaggle/")
    WORKING_DIR = ROOT_DIR / "working/"
    CODE_DIR = ROOT_DIR / "temp/src"
    INPUT_DATA_DIR = ROOT_DIR / "input/action-video/data"
    APPCONFIG_ENV_FILENAME = ".env.kaggle"


elif NOTEBOOK_RUNTIME == "colab":
    from google.colab import drive

    drive.mount("/content/drive")
    SAVE_TO_REMOTE_STORAGE = True

else:
    SAVE_TO_REMOTE_STORAGE = False
    
    # Init root dir once as the working dir can be changed after than. 
    if ROOT_DIR is None:
        ROOT_DIR = (Path.cwd().parent).absolute()
    WORKING_DIR = ROOT_DIR / "kaggle/working/"
    CODE_DIR = ROOT_DIR
    INPUT_DATA_DIR = ROOT_DIR / "kaggle/competitions/action-video/data"
    APPCONFIG_ENV_FILENAME = ".env.local"
        
    

print(f"[Done] Configuring {NOTEBOOK_RUNTIME}")

[Done] Configuring kaggle


In [45]:
print("Working Directory:", WORKING_DIR)
for item in os.listdir(WORKING_DIR):
    print(f"{item}")
    
print("Input Data Directory:", INPUT_DATA_DIR)
for item in os.listdir(INPUT_DATA_DIR):
    print(f"{item}")

Working Directory: /kaggle/working
checkpoints
.virtual_documents
Input Data Directory: /kaggle/input/action-video/data
data_train
test


## Connect to a shared storage

## Download code

In [46]:
# If directory "src" not exist then clone a new one
!pwd
![ -d "{CODE_DIR}" ] || git clone --depth 1  --branch "users/hung-doan/000-update-notebook-log" "https://github.com/aio25-mix002/m07-p7.1" "{CODE_DIR}"

/kaggle/temp/src


## Fetch the latest code

In [47]:
# Go to CODE_DIR, Fetch the latest code
%cd {CODE_DIR}
!git clean -fdx
!git status
!git pull
!pwd


/kaggle/temp/src
Removing build/
Removing logs/
Removing src/__pycache__/
Removing src/lsvit_hmdb51.egg-info/
On branch users/hung-doan/000-update-notebook-log
Your branch is up to date with 'origin/users/hung-doan/000-update-notebook-log'.

nothing to commit, working tree clean
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 779 bytes | 389.00 KiB/s, done.
From https://github.com/aio25-mix002/m07-p7.1
   6138edf..a223acf  users/hung-doan/000-update-notebook-log -> origin/users/hung-doan/000-update-notebook-log
Updating 6138edf..a223acf
Fast-forward
 .env.kaggle  |  3 ++-
 src/utils.py | 10 +++++++++-
 train.py     | 39 +++++++++++++++------------------------
 3 files changed, 26 insertions(+), 26 deletions(-)
/kaggle/temp/src


# Data Preparation

In [48]:
import subprocess
import sys

if NOTEBOOK_RUNTIME != "kaggle":
    # run the script with the same Python interpreter as the notebook
    subprocess.check_call([sys.executable, f"{CODE_DIR}/download_data.py"])


# Train

In [49]:
os.chdir(CODE_DIR)

In [50]:
!uv sync

Using CPython 3.12.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Resolved 161 packages in 3ms
Prepared 91 packages in 44.16s                                           
Installed 91 packages in 448ms                              
 + anyio==4.12.1
 + asttokens==3.0.1
 + beautifulsoup4==4.14.3
 + certifi==2026.1.4
 + charset-normalizer==3.4.4
 + click==8.3.1
 + comm==0.2.3
 + contourpy==1.3.3
 + cycler==0.12.1
 + debugpy==1.8.19
 + decorator==5.2.1
 + executing==2.2.1
 + filelock==3.20.2
 + fonttools==4.61.1
 + fsspec==2025.12.0
 + gdown==5.2.0
 + h11==0.16.0
 + hf-xet==1.2.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + huggingface-hub==1.2.4
 + idna==3.11
 + ipykernel==7.1.0
 + ipython==9.9.0
 + ipython-pygments-lexers==1.1.1
 + ipywidgets==8.1.8
 + jedi==0.19.2
 + jinja2==3.1.6
 + jupyter-client==8.7.0
 + jupyter-core==5.9.1
 + jupyterlab-widgets==3.0.16
 + kagglehub==0.3.13
 + kiwisolver==1.4.9
 + loguru==0.7.3
 + markupsafe==3.0.3
 + matplotlib==3.10.8
 + matplo

In [51]:
if not load_dotenv(dotenv_path= CODE_DIR / f"./{APPCONFIG_ENV_FILENAME}"):
    print(f"Warning: No {APPCONFIG_ENV_FILENAME} file found in {CODE_DIR}")
else:
    print("Loaded the environment variables")

Loaded the environment variables


In [52]:
!python -u ./train.py

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [ ]:
from src.config import ModelConfig, TrainingConfig
from src.utils import find_latest_checkpoint

t_cfg = TrainingConfig()
train_saved_chk_dir: str = find_latest_checkpoint(t_cfg.checkpoint_dir, t_cfg.expr_name)
print("Latest Checkpoint Location:", train_saved_chk_dir)

# Submission

## Load model

In [64]:
import torch
from src.model import LSViTForAction

# Config
m_cfg = ModelConfig()
inf_model = LSViTForAction(config=m_cfg)

# Load the model checkpoint
checkpoint = torch.load(
    os.path.join(train_saved_chk_dir, "best_model.pth"), map_location=t_cfg.device
)
inf_model.load_state_dict(checkpoint['model'])
inf_model = inf_model.to(t_cfg.device)  # Move model to device
inf_model.eval()
print(f"Model loaded: trained acc: {checkpoint['val_acc']:.4f}; {len(checkpoint['train_classes'])} classes")


Model loaded: trained acc: 0.3131; 51 classes


## Load dataset

In [ ]:
from src.dataset import TestDataset
from src.config import TestConfig
from torch.utils.data import DataLoader

test_cfg = TestConfig()
print("\nLoading test dataset...")
test_dataset = TestDataset(
    test_cfg.data_root,
    num_frames=t_cfg.num_frames,
    frame_stride=t_cfg.frame_stride,
    image_size=m_cfg.image_size,
)
test_loader = DataLoader(
    test_dataset,
    batch_size=t_cfg.batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
)
print(f"Test samples: {len(test_dataset)}")

KeyboardInterrupt: 

## Inference

In [66]:
from tqdm.auto import tqdm

print("\nRunning inference...")
predictions = []
checkpoint_classes: list[str] = checkpoint['train_classes']
with torch.no_grad():
    for videos, video_ids in tqdm(test_loader, desc="Inference"):
        videos = videos.to(t_cfg.device)
        logits = inf_model(videos)
        preds = logits.argmax(dim=1)
        for video_id, pred_idx in zip(video_ids.cpu().numpy(), preds.cpu().numpy()):
            pred_class = checkpoint_classes[pred_idx]
            predictions.append((video_id, pred_class))

predictions.sort(key=lambda x: x[0])
print(f"\nTotal predictions: {len(predictions)}")


Running inference...


Inference:   0%|          | 0/64 [00:00<?, ?it/s]


Total predictions: 510


## Save submission

In [ ]:
submission_name = (
    f"submission_vitb{m_cfg.patch_size}"
    f"_d{m_cfg.depth}h{m_cfg.num_heads}"
    f"_smif{m_cfg.smif_window}"
    f"_lr{t_cfg.lr}"
    f"_bs{t_cfg.batch_size}"
    f"_f{t_cfg.num_frames}s{t_cfg.frame_stride}"
    f"_dr{m_cfg.drop_rate}"
    f"_e{checkpoint['metrics']['epoch']}"
    f"_acc{checkpoint['val_acc']:.4f}"
    f".csv"
)
submission_path = train_saved_chk_dir / Path(submission_name)
with open(submission_path, "w") as f:
    f.write("id,class\n")
    for video_id, pred_class in predictions:
        f.write(f"{video_id},{pred_class}\n")

print("=" * 40)
print(f"Submission saved to: {submission_path}")


Submission saved to: /kaggle/working/checkpoints/20260108_165037_default_expr/submission.csv
